# บทที่ 3 การปรับปรุงโครงข่ายประสาทเทียม

จากที่ได้เกริ่นนำในบทที่ผ่านมาว่าการออกแบบและใช้งานโครงข่ายประสาทเทียมในทางปฏิบัติจะเป็นกรรมวิธีที่วนรอบ 
ในช่วงเริ่มต้นของโครงการคงเป็นไปไม่ได้ที่จะกำหนดค่าของไฮเปอร์พารามิเตอร์ต่างๆ ได้อย่างเหมาะสมในทันที เช่นจำนวนชั้นแฝง 
จำนวนเซลล์ในชั้นเหล่านั้น อัตราการเรียนรู้ ฟังก์ชันกระตุ้น ฯลฯ แต่จะต้องผ่านการทดสอบและปรับแต่งแก้ไขจนกว่าจะได้ความแม่นยำตามเป้าหมาย 
ในยุคสมัยนี้ที่ทุกสิ่งทุกอย่างต้องดำเนินการแข่งกับเวลา การศึกษาเกี่ยวกับการปรับแต่งไฮเปอร์พารามิเตอร์จึงเป็นสิ่งที่จำเป็น
เพื่อว่าเราจะทดสอบและแก้ไขเป็นจำนวนรอบน้อยที่สุด ดังนั้นเนื้อหาในบทนี้จะเน้นการปรับแต่งโมเดล DNN จากบทที่ 2 เพื่อให้มีประสิทธิภาพตามต้องการ 
โดยข้อกำหนดของแต่ละโจทย์ปัญหาอาจแตกต่างกันไปตามสมรรถนะของตัวประมวลผลและทรัพยากร จำนวนข้อมูลการฝึก เป็นต้น

ก่อนที่จะกล่าวถึงการปรับปรุงโมเดล DNN ปัจจัยสำคัญอันดับแรกที่ต้องพิจารณาคือการบริหารข้อมูลอย่างไรให้เหมาะสม

## 3.1 การจัดการข้อมูล

เราได้เข้าใจแล้วว่าข้อมูลมีความสำคัญอย่างยิ่งยวดต่อการเรียนรู้เชิงลึก เพราะโมเดล DNN มิใช่ขั้นตอนวิธีที่เขียนเป็นคำสั่งเงื่อนไขตายตัว 
แต่จะถูกขับโดยข้อมูล คือการปรับค่าพารามิเตอร์ของโมเดลในขั้นตอนของการฝึกจนกระทั่งได้ความแม่นยำตามเกณฑ์ที่กำหนด ดังนั้นกล่าวได้ว่า
ประสิทธิภาพของโมเดลขึ้นกับคุณภาพและปริมาณของข้อมูลอย่างมาก 

แนวทางการจัดการข้อมูลสำหรับการเรียนรู้เชิงลึกโดยทั่วไปจะแบ่งข้อมูลเป็น 3 ชุดหลักคือ
1. ชุดข้อมูลสำหรับการฝึก (training set) ใช้ในขั้นตอนการฝึกตามแผนภาพการคำนวณข้างหน้าและย้อนกลับที่ได้อธิบายในบทที่ 2 
โดยมีการวนรอบตามที่กำหนดเพื่อลดค่าสูญเสียและเพิ่มความแม่นยำ 
2. ชุดข้อมูลสำหรับการพัฒนา (development set) คือข้อมูลส่วนหนึ่งที่แยกไว้สำหรับตรวจสอบข้าม (cross validation) เพื่อปรับปรุงโมเดล 
หรือตรวจสอบว่าโมเดลใดให้ผลดีสุดกรณีมีหลายโมเดล
3. ขุดข้อมูลสำหรับทดสอบ (test set) ใช้ในการทดสอบขั้นสุดท้ายเมื่อได้โมเดลที่พอใจแล้ว

ในระบบขนาดเล็กที่ไม่ซับซ้อนมาก เพื่อความง่ายอาจรวมชุดข้อมูล 2,3 เข้าด้วยกันและเรียกเป็นชุดข้อมูลสำหรับทดสอบ 

การแบ่งข้อมูลทั้งหมดออกเป็นชุด 1,2,3 ขึ้นอยู่กับผู้ใช้ นิยมตั้งเป็นอัตราส่วนเช่น 60%, 20%, 20% เช่นมีข้อมูลจำนวน 100,000 ตัวอย่าง 
แบ่งเป็นใช้สำหรับฝึก 60,000 ตัวอย่าง สำหรับพัฒนาและทดสอบอย่างละ 20,000 ตัวอย่าง เป็นต้น ในกรณีที่มีข้อมูลจำนวนมาก เช่นล้านตัวอย่าง 
อาจลดอัตราส่วนชุดข้อมูลพัฒนาและทดสอบน้อยลงเช่นเหลือเพียงอย่างละ 1% 

ข้อควรระวังเกี่ยวกับการใช้ชุดข้อมูลฝึกและทดสอบคือ ข้อมูลต้องมาจากการแจกแจง (distribution) ที่เหมือนกัน 
ตัวอย่างของการใช้ชุดข้อมูลที่ไม่สอดคล้องกันคือ สมมุติว่าต้องการสร้างโมเดลจำแนกสุนัขและแมว เราค้นหาข้อมูลภาพจากอินเทอร์เน็ตเพื่อฝึกโมเดล 
ซึ่งเป็นภาพสัตว์เลี้ยงที่ส่วนใหญ่ถ่ายโดยช่างภาพมืออาชีพ มีความชัดเจนและรายละเอียดสูง
แต่เมื่อนำไปทดสอบหรือใช้งานจากภาพผู้ใช้ที่อาจไม่มีคุณภาพดีเท่าเทียม เช่นเบลอ แสงน้อย จัดองค์ประกอบภาพไม่ดี ผลที่ได้ก็คงไม่เป็นตามที่ต้องการ 

### 3.1.1 ค่าเอนเอียงและความแปรปรวน

รูปที่ 3.1 แสดงตัวอย่างการจำแนกข้อมูลในระนาบ 2 มิติ โดยข้อมูลสองประเภทแทนด้วยวงกลมและสี่เหลี่ยม ในรูป (a) ด้านซ้ายสุดเป็นการจำแนกโดยเส้นตรง
จะเห็นว่าไม่สามารถแบ่งกลุ่มข้อมูลได้ดีเท่าที่ควร คือมีการฟิตตำ่ไป (underfit) เรียกกรณีนี้ว่า มีค่าเอนเอียงสูง (high bias) 

<p />  
<p align="center">
<img src="https://drive.google.com/uc?id=17vghyzLUAh4xuVnZMRvSxEEmOY0nn8r6" width=750 />
</p>
<div align="center">รูปที่ 3.1 การจำแนกทวิภาคในระนาบ 2 มิติ</div>


**หมายเหตุ :** คำว่าไบแอสหรือค่าเอนเอียงในบริบทนี้ไม่มีความเกี่ยวข้องกับค่าเอนเอียง b ที่เป็นพารามิเตอร์ของโมเดล

หากพิจารณารูป (c) ด้านขวาสุดซึ่งเป็นการฟิตเกิน (overfit) ถึงแม้ว่าจะสามารถจำแนกข้อมูลฝึกได้อย่างสมบูรณ์แต่ก็มีความซับซ้อนเกินความจำเป็น
ไม่สามารถคาดหวังว่าจะทำงานได้ดีกับข้อมูลทดสอบหรือข้อมูลใหม่ที่เข้ามา เรียกกรณีนี้ว่า มีความแปรปรวนสูง (high variance) 
ตัวจำแนกที่อยู่ระหว่างกลางคือรูป (c) ที่เป็นตัวเลือกที่เหมาะสมสำหรับปัญหานี้ กรณีเหล่านี้จะเกิดขึ้นเช่นเดียวกันในระบบหลายมิติ 
การยกตัวอย่างในระนาบ 2 มิติทำให้เห็นภาพได้ชัดเจนขึ้นเท่านั้น 

สาระสำคัญของเนื้อหาในหัวข้อย่อยนี้คือ 
ต้องการหลีกเลี่ยงทั้งกรณีค่าเอนเอียงสูงและความแปรปรวนสูง ในสมัยก่อนจะมีประเด็นที่นิยมกล่าวถึงกันเรียกว่า 
การประนีประนอมระหว่างค่าเอนเอียงและความแปรปรวน (bias-variance tradeoff) 
เนื่องจากการปรับค่าเอนเอียงลดลงมีผลกระทบทำให้ความแปรปรวนสูงขึ้น หรือกลับกัน แต่ในยุคของการเรียนรู้เชิงลึกจะถูกกล่าวถึงน้อยลง 
ทั้งนี้เนื่องจากการใช้โมเดลที่มีขนาดใหญ่สามารถช่วยลดค่าเอนเอียงโดยไม่กระทบความแปรปรวน 
ในทำนองเดียวกันการเพิ่มขนาดของข้อมูลช่วยลดความแปรปรวนได้โดยไม่กระทบกับค่าเอนเอียง

รูปที่ 3.2 แสดงตัวอย่างการจำแนกทวิภาคภาพสุนัขและแมว โดยโมเดลจะให้เอาต์พุตเท่ากับ 0 สำหรับภาพสุนัขและ 1 สำหรับภาพแมว 
เราจะเปรียบเทียบค่าผิดพลาดของกรณี A,B,C,D กํบกรณีตรวจสอบโดยมนุษย์ โดยสมมุติว่ามนุษย์สามารถจำแนกภาพได้โดยมีความผิดพลาดเป็นศูนย์ 

<p />  
<p align="center">
<img src="https://drive.google.com/uc?id=1CkNZsqNEPCHOYyRsjuCODrLbaWgiNipw" width=500 />
</p>
<div align="center">รูปที่ 3.2 ปัญหาการจำแนกภาพสุนัขและแมว</div>

* A. ได้ค่าผิดพลาดจากชุดข้อมูลฝึกเท่ากับ 0.5% และค่าผิดพลาดจากชุดข้อมูลพัฒนาเท่ากับ 1% จัดได้ว่าเป็นโมเดลที่ดี 
คือได้ทั้งค่าเอนเอียงและความแปรปรวนต่ำ 
* B. ได้ค่าผิดพลาดจากชุดข้อมูลฝึกเท่ากับ 1% และค่าผิดพลาดจากชุดข้อมูลพัฒนาเท่ากับ 12% คือกรณีความแปรปรวนสูง 
จะเห็นว่าโมเดลมีความแม่นยำสูงกับข้อมูลฝึก แต่ทำงานได้ไม่ดีกับข้อมูลพัฒนาเนื่องจากการฟิตเกิน 
* C. ได้ค่าผิดพลาดจากชุดข้อมูลฝึกเท่ากับ 20% และค่าผิดพลาดจากชุดข้อมูลพัฒนาเท่ากับ 21% คือกรณีค่าเอนเอียงสูง 
โมเดลทำงานได้ไม่ดีตั้งแต่กับชุดข้อมูลการฝึก และก็ได้ผลใกล้เคียงกันกับชุดข้อมูลพัฒนา คือมีการฟิตต่ำไป
* D. ได้ค่าผิดพลาดจากชุดข้อมูลฝึกเท่ากับ 20% และค่าผิดพลาดจากชุดข้อมูลพัฒนาเท่ากับ 30% คือกรณีทั้งค่าเอนเอียงและความแปรปรวนสูง 
เป็นโมเดลที่ด้อยที่สุดสำหรับทั้ง 4 กรณีที่ยกตัวอย่างมา

### 3.1.2 วิธีพื้นฐานในการลดค่าเอนเอียงและความแปรปรวน

เมื่อตรวจสอบสมรรถนะของโมเดลที่มีต่อข้อมูลชุดฝึกและพัฒนาดังเช่นในตัวอย่างด้านบน เราสามารถตัดสินเบื้องต้นเกี่ยวกับค่าเอนเอียงและความแปรปรวนของโมเดล 
และใช้วิธีการพื้นฐานในการแก้ปัญหาดังนี้

หากพบว่าโมเดลมีค่าเอนเอียงสูง (ต้องการปรับปรุงสมรรถนะต่อข้อมูลฝึก)
* เพิ่มขนาดของ DNN
* เพิ่มจำนวนรอบการฝึก หรือทดลองเปลี่ยนตัวหาค่าเหมาะที่สุด
* อาจทดลองเปลี่ยนสถาปัตยกรรมของโครงข่ายประสาทเทียม 

หากพบว่าโมเดลมีความแปรปรวนสูง (ต้องการปรับปรุงสมรรถนะต่อข้อมูลพัฒนา)
* เพิ่มจำนวนข้อมูล
* ลดการฟิตเกินโดยวิธีเช่นเรกูลาร์ไรเซชันหรือดรอปเอาต์ (จะได้กล่าวถึงต่อไปในบทนี้)
* อาจทดลองเปลี่ยนสถาปัตยกรรมของโครงข่ายประสาทเทียม 

เมื่อปรับเปลี่ยนโมเดลแล้วต้องกลับไปตรวจสอบใหม่จนกว่าจะได้ค่าเอนเอียงและความแปรปรวนอยู่ในเกณฑ์ที่ยอมรับได้

<p align="center">
<img src="https://drive.google.com/thumbnail?id=13bzT7Rmy3bzvE7TiS0yfQo94kpxMuipF" alt="dewninja"/>
</p>
<div align="center">dew.ninja 2022</div>